In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터를 대화형 데이터로 전처리.

import openpyxl
import json

excel_file_path = '/content/drive/My Drive/Colab-Notebooks/lab/data/original_QA.xlsx'
jsonl_file_path = '/content/drive/My Drive/Colab-Notebooks/lab/data/output.jsonl'

workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

cnt = 0
column_data = []
for row in sheet.iter_rows(min_row=2, values_only=True):
  if cnt == 30:
    break
  top_obj = {}
  top_obj['messages'] = []
  for i in range(0, 3):
    obj = {}
    if(i == 0):
      obj['role'] = 'system'
      obj['content'] = '너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야.'
    elif(i == 1):
      obj['role'] = 'user'
      obj['content'] = str(row[2])
    else:
      obj['role'] = 'assistant'
      obj['content'] = str(row[3])
    top_obj['messages'].append(obj)
  column_data.append(top_obj)
  cnt += 1

with open(jsonl_file_path, 'w', encoding='utf-8') as jsonlfile:
    for item in column_data:
        json.dump(item, jsonlfile, ensure_ascii=False)
        jsonlfile.write('\n')

print(f' 열의 내용이 JSON 파일({jsonl_file_path})로 저장되었습니다.')


 열의 내용이 JSON 파일(/content/drive/My Drive/Colab-Notebooks/lab/data/output.jsonl)로 저장되었습니다.


In [ ]:
import openpyxl
import json

excel_file_path = '/content/drive/My Drive/Colab-Notebooks/lab/data/original_QA.xlsx'
jsonl_file_path = '/content/drive/My Drive/Colab-Notebooks/lab/data/output.jsonl'

workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

cnt = 0
column_data = []
for row in sheet.iter_rows(min_row=2, values_only=True):
  obj = {}
  if cnt == 30:
    break
  obj['prompt'] = str(row[2])
  obj['completion'] = str(row[3])
  column_data.append(obj)
  cnt += 1

with open(jsonl_file_path, 'w', encoding='utf-8') as jsonlfile:
    for item in column_data:
        json.dump(item, jsonlfile, ensure_ascii=False)
        jsonlfile.write('\n')

print(f' 열의 내용이 JSON 파일({jsonl_file_path})로 저장되었습니다.')


 열의 내용이 JSON 파일(/content/drive/My Drive/Colab-Notebooks/lab/data/output.jsonl)로 저장되었습니다.


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
### 데이터 셋이 잘 준비되었는지 확인한다.
# We start by importing the required packages
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

# Next, we specify the data path and open the JSONL file

data_path = "/content/drive/My Drive/Colab-Notebooks/lab/data/output.jsonl"

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")

Num examples: 30
First example:
{'role': 'system', 'content': '너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야.'}
{'role': 'user', 'content': '전립선은 여성에도 있나요? '}
{'role': 'assistant', 'content': '아닙니다. 전립선은 남성에게만 있는 생식기관 입니다'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 88, 300
mean / median: 172.23333333333332, 170.5
p5 / p95: 119.80000000000001, 228.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 24, 225
mean / median: 99.3, 93.5
p5 / p95: 50.0, 158.10000000000002

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~5167 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~15501 tokens
See pricing page to estimate total costs


In [ ]:
!pip install openai

In [ ]:
import openai
import os

openai.api_key = ""

openai.File.create(
  file=open("/content/drive/My Drive/Colab-Notebooks/lab/data/output.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-slSQA8hwXpAVSISwjbvdjEVq at 0x7beaa0164950> JSON: {
  "object": "file",
  "id": "file-slSQA8hwXpAVSISwjbvdjEVq",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 14878,
  "created_at": 1693989096,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
openai.FineTuningJob.create(training_file="file-FaAlAHuBjyCI7s93UQ4qWdzV", model="gpt-3.5-turbo-0613", hyperparameters={"n_epochs":3})

<FineTuningJob fine_tuning.job id=ftjob-JNFXsKlYY3rwZ0fglne5oqP7 at 0x7f542b8a9df0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-JNFXsKlYY3rwZ0fglne5oqP7",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693979314,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-ySXADTYxkv77kxJzfGrXW0be",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-FaAlAHuBjyCI7s93UQ4qWdzV",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [ ]:
openai.FineTuningJob.create(training_file="file-FaAlAHuBjyCI7s93UQ4qWdzV", model="gpt-3.5-turbo-0613", hyperparameters={"n_epochs":5})

<FineTuningJob fine_tuning.job id=ftjob-tDuAUyaAdgrOYEk9K6cjFyFj at 0x7bea7f2a1a80> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-tDuAUyaAdgrOYEk9K6cjFyFj",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693989540,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-ySXADTYxkv77kxJzfGrXW0be",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-FaAlAHuBjyCI7s93UQ4qWdzV",
  "hyperparameters": {
    "n_epochs": 5
  },
  "trained_tokens": null
}

In [ ]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7bea7f3bb150> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-tDuAUyaAdgrOYEk9K6cjFyFj",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693989540,
      "finished_at": 1693990210,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7viWhUBa",
      "organization_id": "org-ySXADTYxkv77kxJzfGrXW0be",
      "result_files": [
        "file-7WG6zhoXS9f0miD7WGx2ma1k"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-FaAlAHuBjyCI7s93UQ4qWdzV",
      "hyperparameters": {
        "n_epochs": 5
      },
      "trained_tokens": 25535
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-JNFXsKlYY3rwZ0fglne5oqP7",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693979314,
      "finished_at": 1693979983,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7vfrjEpe",
      "organization_id": "org-ySXADTYxkv77kxJzfGrXW0be",
      

In [ ]:
openai.FineTuningJob.retrieve("ftjob-JNFXsKlYY3rwZ0fglne5oqP7")

<FineTuningJob fine_tuning.job id=ftjob-JNFXsKlYY3rwZ0fglne5oqP7 at 0x7f542ae02660> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-JNFXsKlYY3rwZ0fglne5oqP7",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693979314,
  "finished_at": 1693979983,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7vfrjEpe",
  "organization_id": "org-ySXADTYxkv77kxJzfGrXW0be",
  "result_files": [
    "file-UErpSMa7glOPcYV6J7R8stq8"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-FaAlAHuBjyCI7s93UQ4qWdzV",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 15321
}

In [ ]:
# 파인튜닝 작업 10개 나열
openai.FineTuningJob.list(limit=10)

# 파인튜닝 작업의 상태 검색
openai.FineTuningJob.retrieve("ftjob-BZgsjMC8x49lwyt3jifqht5d")

# 작업 취소
openai.FineTuningJob.cancel("ft-abc123")

# 파인튜닝 작업에서 최대 10개 이벤트 나열
openai.FineTuningJob.list_events(id="ft-abc123", limit=10)

# 파인튜닝된 모델 삭제 (모델이 생성된 조직의 소유자여야 함)
openai.Model.delete("ft-abc123")

In [ ]:
openai.FineTuningJob.list_events(id="ftjob-JNFXsKlYY3rwZ0fglne5oqP7", limit=10)

<OpenAIObject list at 0x7bea7fffbb00> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-AkeuAhWXtfyK311hH653EFy0",
      "created_at": 1693979985,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-bdhuRZZ6IPwHKWg0pJV4JUix",
      "created_at": 1693979984,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::7vfrjEpe",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-h8XhwM0tHoVFsehrXjSzRXq2",
      "created_at": 1693979977,
      "level": "info",
      "message": "Step 90/90: training loss=0.59",
      "data": {
        "step": 90,
        "train_loss": 0.5945106744766235,
        "train_mean_token_accuracy": 0.8571428656578064
      },
      "type": "metrics"
  

In [ ]:
# 5에폭
completion2 = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::7viWhUBa",
  messages=[
    {"role": "system", "content": "너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야."},
    {"role": "user", "content": "인종에 따른 전립선암의 발생의 차이에 대해서 알려주세요."}
  ],
  temperature=0.2,
  max_tokens=256,
)

print(completion2.choices[0].message["content"])

한국과 비교하여 미국에서는 흑인인종에서 전립선암의 발생률이 백인인종에 비해 1.6배 정도 높게 나타나고 있습니다. 


In [ ]:
# 3에폭
completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::7vfrjEpe",
  messages=[
    {"role": "system", "content": "너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야."},
    {"role": "user", "content": "인종에 따른 전립선암의 발생의 차이에 대해서 알려주세요."}
  ],
  temperature=0.2,
  max_tokens=256,
)

print(completion.choices[0].message["content"])

인종에 따른 전립선암의 발생률은 인종에 따라 차이가 있습니다. 전립선암은 서양인과 비교하여 아시아인에서는 낮은 발생률을 보이며, 흑인에서는 서양인에 비해 높은 발생률을 보입니다.


In [ ]:
# 5에폭
completion2 = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::7viWhUBa",
  messages=[
    {"role": "system", "content": "너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야."},
    {"role": "user", "content": "인종에 따른 전립선암의 발생의 차이에 대해서 알려주세요."}
  ],
  temperature=0.2,
  max_tokens=256,
)

print(completion2.choices[0].message["content"])

한국에서는 전립선암의 발생률이 서구 국가에 비해 낮은 편이지만, 서구 국가에 비해 전립선암의 발생연령이 젊고, 발생률이 급격히 증가하고 있습니다. 특히, 한국인 남성에게 전립선암은 악성종양 중 가장 많이 발생하는 암으로 알려져 있습니다. 2020년 국가암등록통계에 따르면, 전립선암은 남성 암 발생의 9.6%를 차지하여, 1위 암으로 발생하였습니다.


In [ ]:
# 3에폭
completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::7vfrjEpe",
  messages=[
    {"role": "system", "content": "너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야."},
    {"role": "user", "content": "사랑니 발치 후 주의사항에 대해서 알려줘."}
  ],
  temperature=0.2,
  max_tokens=256,
)

print(completion.choices[0].message["content"])

사랑니 발치 후에는 다음과 같은 주의사항을 지켜야 합니다.
1. 출혈 : 수술 후 2~3시간 동안은 출혈이 심할 수 있으므로, 입안에 누는 거즈를 30분 동안 계속 누워 주세요.
2. 통증 : 수술 후 2~3일 동안 통증이 심할 수 있으므로, 처방 받은 진통제를 규칙적으로 복용하세요.
3. 종창 : 수술 후 2~3일 동안은 종창이 심할 수 있으므로, 냉찜질을 하거나 얼음을 입에 대지 마세요.
4. 식사 : 수술 후 2~3일 동안은 고기류나 딱딱한 음식은 피하고, 액


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
    {"role": "system", "content": "너는 의학적 지식을 기반으로 질문에 대해 답하는 챗봇이야."},
    {"role": "user", "content": "사랑니 발치 후 주의사항에 대해서 알려줘."}
  ],
  temperature=0.2,
  max_tokens=256,
)
print(response.choices[0].message["content"])

사랑니 발치 후에는 몇 가지 주의사항을 지켜야 합니다. 다음은 일반적인 주의사항입니다:

1. 충분한 휴식: 수술 후에는 몸을 휴식시키고 힘들게 하지 않는 것이 중요합니다. 수술 후 24시간 동안은 휴식을 취하고, 활동을 최소화해야 합니다.

2. 통증 관리: 수술 후에는 통증이 발생할 수 있습니다. 의사가 처방한 통증 완화제를 규칙적으로 복용하고, 얼음 팩을 사용하여 통증을 완화시킬 수 있습니다.

3. 혈액 응고: 수술 후에는 혈액 응고가 발생할 수 있으므로, 의사의 지시에 따라 혈전 예방을 위


In [ ]:
response3 = openai.Completion.create(
  model="ft:babbage-002:personal::7vVbKlAL",
  prompt="인종에 따른 전립선암의 발생의 차이에 대해서 알려주세요.",
  max_tokens=256,
)

print(response3.choices[0].text)

 전립선암의 종사와 전립선암의 전기적 정책이 다르다고 하면, 전립선암의 독특성은 아래와 같습니다. 전립선암의 종사는 유전자전류, 유전자안전, 우정, 스테인레스 액체 등 다양한 전립선암질(정도촉진기 등)을 접목하여 발생합니다. 전립선암의 전기적 정책은 세폭기간에서 최대 2단계를 동시에 전립선암에서 갖춘 전력장애를 경험하고 진행합니다. 전립선암의 갖춘 전력장애는 70달이상 높습니다. 세폭기간에 7일 새로 인해 전립선암의 갖춘 전력은
